<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc" style="margin-top: 1em;"><ul class="toc-item"><li><span><a href="#Dowload-GTFS-tables-from-google-sheets" data-toc-modified-id="Dowload-GTFS-tables-from-google-sheets-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Dowload GTFS tables from google sheets</a></span></li><li><span><a href="#Generate-stop_times-simplified" data-toc-modified-id="Generate-stop_times-simplified-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Generate stop_times simplified</a></span></li><li><span><a href="#Calculate-stop-distances" data-toc-modified-id="Calculate-stop-distances-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Calculate stop distances</a></span></li><li><span><a href="#Calculate-time-interval-between-stops" data-toc-modified-id="Calculate-time-interval-between-stops-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Calculate time interval between stops</a></span></li><li><span><a href="#Complete-stop_times" data-toc-modified-id="Complete-stop_times-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Complete stop_times</a></span></li></ul></div>

In [315]:
%reload_ext autoreload
%autoreload 2

import pandas as pd
import datetime
from copy import deepcopy
from pathlib import Path
import folium as f
import geopandas as gpd

In [316]:
import gtfs_kit as gk

In [317]:
gtfs_path = '../data/raw/gtfs_brt'
tempo_ciclo_path = f'../data/raw/gtfs_brt/tempo_ciclo_brt.csv'
stop_times_path = '../data/raw/gtfs_brt/stop_times.txt'

# Dowload GTFS tables from google sheets

In [318]:
sheets_url = 'https://docs.google.com/spreadsheets/d/{key}/gviz/tq?tqx=out:csv&sheet={sheet_name}'
key = '1_0CaioUIayDCTPDS5KnS9pFpcAgP_7bpkhDyr_Z2stI'
sheets = ['agency', 'trips', 'calendar', 'routes', 'stops', 'stop_sequences', 
          'frequencies', 'shapes']

In [319]:
for sheet in sheets:
    df = (pd.read_csv(sheets_url.format(key=key, sheet_name=sheet), dtype=str)
    .applymap(lambda x: x.replace(',', '.') if isinstance(x, str) else x)
    .dropna(how='all', axis=1))
    df.to_csv(
        Path(gtfs_path) / (sheet + '.txt'), index=False
    )

# Generate stop_times simplified

In [320]:
tables = ['trips', 'stop_sequences', 'frequencies']

gtfs = {table: pd.read_csv(f'../data/raw/gtfs_brt/{table}.txt') for table in tables}
all_trip_ids = gtfs['trips'][['route_id', 'direction_id', 'trip_id']]

(gtfs['stop_sequences'].replace([1,2], [0,1])
 .merge(all_trip_ids, on=['route_id', 'direction_id'])
 [['trip_id', 'stop_id', 'stop_sequence']]
 .assign(departure_time=None)
 .assign(arrival_time=None) 
 .to_csv(stop_times_path, index=False))

# Calculate stop distances

In [321]:
feed = gk.read_feed(gtfs_path, dist_units='km')

In [322]:
print('Frequencies ', get_difference(feed.trips, feed.frequencies, 'trip_id'))
print('Stop Times', get_difference(feed.trips, feed.stop_times, 'trip_id'))
print('Frequencies', get_difference(feed.trips, feed.frequencies, 'trip_id'))
# print('Stop Sequences', get_difference(feed.trips, gtfs['stop_sequences'], 'trip_id'))

Frequencies  set()
Stop Times set()
Frequencies set()


In [323]:
stop_times = feed.append_dist_to_stop_times().stop_times

In [324]:
stop_times[stop_times['shape_dist_traveled'].isna()]['trip_id'].unique()

array(['200042A111020001'], dtype=object)

# Calculate time interval between stops

In [325]:
tempo_ciclo = pd.read_csv(tempo_ciclo_path).dropna(subset=['direction_id'])
tempo_ciclo = (
pd.merge(
    gtfs['stop_sequences'][['route_id_old_old', 'route_id']],
    tempo_ciclo,
    left_on='route_id_old_old', right_on='route_id',)
.drop(['route_id_y', 'route_id_old_old'], 1)
.rename(columns={'route_id_x': 'route_id'})
              )


In [326]:
tempo_ciclo = pd.merge(
    feed.trips.replace([1,2], [0,1])[['route_id', 'direction_id', 'trip_id']],
    tempo_ciclo,
    on=['route_id', 'direction_id']).drop_duplicates()

In [327]:
stop_times = stop_times.merge(tempo_ciclo[['trip_id', 'tempo_ciclo']], on='trip_id')

In [328]:
stop_times['norm_dist'] = stop_times.groupby('trip_id').transform(lambda x: x/x.max())['shape_dist_traveled']
stop_times['time_btw_stops'] = stop_times['norm_dist'] * stop_times['tempo_ciclo'] / 2
stop_times = stop_times.dropna(subset=['time_btw_stops'])

In [329]:
def get_difference(df1, df2, column):
    
    return set(df1[column]).difference(set(df2[column].unique()))

In [330]:
get_difference(feed.trips,  stop_times, 'trip_id')

{'200042A111020001'}

# Complete stop_times

In [331]:
def add_departure(df):
    
    departures = []
    start_time = pd.Timestamp('2000-01-01 01:00:00')

    for i, stops in stop_times[stop_times['trip_id'] == df['trip_id']].iterrows():

        if stops['stop_sequence'] == 0:

            stop_time = start_time

        else:

            stop_time = start_time + datetime.timedelta(minutes=stops['time_btw_stops'])

        departures.append(deepcopy({
            'departure_time': deepcopy(stop_time),
            'stop_sequence': stops['stop_sequence'],
            'stop_id': stops['stop_id']
        }))

        
    return (pd.DataFrame(departures)
            .assign(trip_id=df['trip_id'])
           )

In [332]:
gtfs['frequencies']['start_time'] = gtfs['frequencies']['start_time'].apply(lambda x: datetime.datetime.strptime(x,'%H:%M:%S'))
gtfs['frequencies']['end_time'] = gtfs['frequencies']['end_time'].apply(lambda x: datetime.datetime.strptime(x,'%H:%M:%S'))

In [333]:
departures = pd.concat(
         gtfs['frequencies']
         .drop_duplicates(subset=['trip_id'])
         .apply(add_departure, 1)
         .to_list()
        ).reset_index(drop=True)

departures['departure_time'] = departures['departure_time'].apply(lambda x: x.strftime('%H:%M:%S'))
departures['arrival_time'] = departures['departure_time']
departures['stop_sequence'] = departures['stop_sequence'].astype(int)
departures[['trip_id', 'arrival_time', 'departure_time', 'stop_id', 'stop_sequence']
          ].to_csv(stop_times_path, index=False)

In [334]:
get_difference(feed.trips, gtfs['frequencies'], 'trip_id')

set()

In [335]:
trips = set(gtfs['trips']['trip_id'].unique())
print('Missing trip_ids: ', trips.difference(set(departures['trip_id'].unique())))

Missing trip_ids:  {'200042A111020001'}


In [336]:
# Validate

In [337]:
feed = gk.read_feed(gtfs_path, dist_units='km')

In [338]:
feed.validate()

,type,message,table,rows
2,error,Headway periods for the same trip overlap,frequencies,[4]
3,error,Headway periods for the same trip overlap,frequencies,[0]
4,error,Headway periods for the same trip overlap,frequencies,[0]
5,error,Headway periods for the same trip overlap,frequencies,[0]
0,warning,Unrecognized column old_trip_id,frequencies,[]
1,warning,Unrecognized column calendar,frequencies,[]
6,warning,Unrecognized column old_route_id,routes,[]
7,warning,Unrecognized column old_old_shape_id,shapes,[]
8,warning,Unrecognized column old_shape_id,shapes,[]
12,warning,"Repeated pair (trip_id, departure_time)",stop_times,"[1305, 1306, 1307, 1308, 1309, 1310, 1311, 131..."


In [339]:
feed.write('../data/output/gtfs_brt.zip')

In [340]:
feed.stop_times.query('trip_id == "200053A113012101"')

,trip_id,arrival_time,departure_time,stop_id,stop_sequence
1490,200053A113012101,01:00:00,01:00:00,533135204000191,0
1491,200053A113012101,01:04:14,01:04:14,533137204000172,1
1492,200053A113012101,01:10:12,01:10:12,416115204000172,3
1493,200053A113012101,01:14:17,01:14:17,424128203000472,4
1494,200053A113012101,01:33:47,01:33:47,424128203000372,5
1495,200053A113012101,01:35:00,01:35:00,424128201001092,6


In [313]:
stop_times.query('trip_id == "200053A113012101"')

,trip_id,stop_id,stop_sequence,departure_time,arrival_time,shape_dist_traveled,tempo_ciclo,norm_dist,time_btw_stops
1529,200053A113012101,533135204000191,0,NaN,NaN,0.036793,70,0.001964,0.068751
1530,200053A113012101,533137204000172,1,NaN,NaN,2.266674,70,0.121014,4.235492
1531,200053A113012101,416115204000172,3,NaN,NaN,5.462495,70,0.291634,10.207185
1532,200053A113012101,424128203000472,4,NaN,NaN,7.650588,70,0.408453,14.295841
1533,200053A113012101,424128203000372,5,NaN,NaN,18.088129,70,0.965696,33.799367
1534,200053A113012101,424128201001092,6,NaN,NaN,18.730662,70,1.000000,35.000000
